In [39]:
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
import torch
from transformers import BertTokenizer, BertModel
import pickle

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', output_hidden_states = True)

dataset = pd.read_csv("IMDB_dataset_untouched.csv")
dataset.drop(dataset.tail(49900).index, inplace = True)
reviews = [sent_tokenize(w) for w in dataset["review"]]
flattened_reviews = [item for sublist in reviews for item in sublist]

In [40]:
encodings = tokenizer(flattened_reviews, padding = 'max_length', truncation = True, max_length = 512)

In [41]:
#encodings["input_ids"]

In [42]:
tokenized_text = [tokenizer.convert_ids_to_tokens(i) for i in encodings["input_ids"]]

In [43]:
#tokenized_text

In [44]:
tokens_tensor = torch.tensor(torch.LongTensor(encodings["input_ids"]))
segments_tensors = torch.tensor(encodings["token_type_ids"])

C:\Users\78mik\AppData\Local\Temp\ipykernel_13676\2413880606.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens_tensor = torch.tensor(torch.LongTensor(encodings["input_ids"]))


In [45]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [46]:
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]

hidden_states

(tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
          [ 0.5812,  0.6695,  0.1027,  ...,  0.2468,  0.3671,  0.1771],
          [-0.1866,  0.2322, -0.5005,  ..., -0.4061, -1.0823, -0.5774],
          ...,
          [ 0.6458, -0.5409, -0.1780,  ..., -0.0482, -0.3466, -0.4825],
          [ 0.7417, -0.7271,  0.3278,  ..., -0.2011, -0.6038, -0.4935],
          [ 0.2992, -1.0338,  0.1294,  ...,  0.2149,  0.2113, -1.5097]],
 
         [[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
          [ 1.0008, -0.2777, -0.8142,  ...,  0.0639, -0.7440, -0.2539],
          [-0.4019,  0.9467,  0.2703,  ..., -0.1535, -0.3445, -0.1230],
          ...,
          [ 0.6458, -0.5409, -0.1780,  ..., -0.0482, -0.3466, -0.4825],
          [ 0.7417, -0.7271,  0.3278,  ..., -0.2011, -0.6038, -0.4935],
          [ 0.2992, -1.0338,  0.1294,  ...,  0.2149,  0.2113, -1.5097]],
 
         [[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
          [ 0.0726,  0.8685,

In [47]:
pickle_out = open("bert_vectors_untouched.pickle", "wb")
pickle.dump(hidden_states, pickle_out)
pickle_out.close()

In [48]:
len(hidden_states[0][0])

512

In [49]:
# hidden_states[0] contains the reviews
len(hidden_states[0])

100

In [50]:
# hidden_states[0][0] contains the reviews
hidden_states[0][0]

tensor([[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
        [ 0.5812,  0.6695,  0.1027,  ...,  0.2468,  0.3671,  0.1771],
        [-0.1866,  0.2322, -0.5005,  ..., -0.4061, -1.0823, -0.5774],
        ...,
        [ 0.6458, -0.5409, -0.1780,  ..., -0.0482, -0.3466, -0.4825],
        [ 0.7417, -0.7271,  0.3278,  ..., -0.2011, -0.6038, -0.4935],
        [ 0.2992, -1.0338,  0.1294,  ...,  0.2149,  0.2113, -1.5097]])

In [51]:
hidden_states[0][0][1]

tensor([ 5.8122e-01,  6.6948e-01,  1.0267e-01, -9.3402e-02,  3.7572e-01,
         3.2371e-01,  6.0433e-01,  4.0796e-01,  8.0506e-01, -2.4809e-02,
         4.1792e-01,  1.5264e-01,  1.6942e-01, -1.7260e-01, -3.9346e-01,
         2.5298e-01,  6.4843e-01,  5.5470e-01, -6.0719e-01,  8.6400e-01,
         2.3630e-01, -2.1402e-01, -5.6126e-01,  7.6187e-01,  6.0718e-01,
         1.7897e-01, -4.4566e-02,  7.9816e-02,  2.8847e-01,  8.5886e-01,
         2.8395e-01, -1.1083e+00, -1.3384e-01, -2.6143e-02,  1.3806e-01,
         8.5364e-02,  7.2336e-01, -4.1928e-01, -1.7518e-02,  1.2012e+00,
        -1.8454e-01, -6.7315e-02, -7.9677e-01, -2.1576e-01, -2.1443e-01,
        -2.2820e-01,  2.7453e-01,  7.9882e-02, -3.5037e-01, -6.2966e-02,
        -5.8613e-01, -1.7123e-01, -1.1035e+00, -2.1615e-01,  6.4415e-01,
        -5.2553e-01,  4.2121e-01, -2.1900e-01, -7.5691e-01, -3.8401e-01,
         1.0405e+00, -6.1051e-02, -8.4628e-01, -1.0133e+00,  3.2911e-01,
        -1.4960e-01,  7.4285e-01,  2.5884e-01, -4.6